In [1]:
import pandas as pd

## Creating variables
### human input
equip - equipment types

cube_name - name of the cube <b>this will override prime_rates</b>

desired_lines(tuple) - starts with lines that is needed <br>
    e.g. Critical in Critical rate/Damage

desired_number - number of desired lines from desired_lines

prime_rates - the chance of priming each line a tuple of any length 

Tiers (tuple) - the rank of equipment and sub line Currently only support legend with unique sub

main_line - string that shoes the rank of first line

additional - boolean to check if equip is add pot

### generated

sub_line - string that generate the second line

column_header(list) - generated from  varaible prime_rates the header in output csv

Lines (dicctionary) - the possible lines that could come out <br>
    keys - elements in Tiers
    value- dataframes from read csv in lines weight format

In [2]:
equip=r"Weapon"
cube_name="Additional" #leave it as a blank string if you are lanning to input prime_rates
prime_rates=(100,00,0) # Can put None if cube_name and main_line is given
desired_lines=("ATT")
desired_number=2
main_line="Legend" # "Epic", "Unique", "Legend"
additional = True
column_header= [] # ["line1", "line2", "line3",...,"desired"]
for x in range(len(prime_rates)):
    column_header.append("line"+ str(x+1))
column_header.append("desired")
Tiers= ("Rare", "Epic", "Unique", "Legend")
sub_line=Tiers[Tiers.index(main_line)-1]
Lines={}

In [3]:
if cube_name!='':
    df=pd.read_csv("Resources/prime rates.csv", index_col=0, na_values='')
    x=df.loc[:,main_line][cube_name]
    assert x!='-', "can't cube {0} equip with {1} cube".format(main_line,cube_name)
    prime_rates=tuple(map(float, x.split(',')))
    print(prime_rates)

(100.0, 0.4975, 0.4975)


## Accessing data frame

df.columns # shows header

df.Line or df.weight # shoes specific columns


In [4]:
"""
Checking the dataframe in Lines
"""
for i in (main_line,sub_line):
    try:
        Lines[i]=pd.read_csv("Resources/{0}/{1}{2}.csv".format(equip,"Add_" if additional else "" ,i))
    except FileNotFoundError:
        print("No records for",i, equip,)
print(Lines[sub_line].shape) # shows all number of lines
print(Lines[main_line].shape)


(19, 2)
(20, 2)


In [5]:
"""
printing possible lines
"""
for x in Lines.items():
    print(x[0])
    print(list(x[1].Line))

Legend
['Max HP +10%', 'Max MP +10%', 'ATT +12%', 'MATT +12%', 'Critical Chance +12%', 'STR +12%', 'DEX +12%', 'INT +12%', 'LUK +12%', 'Damage +12%', 'All Stats +9%', 'STR +2 per 10 Character Levels', 'DEX +2 per 10 Character Levels', 'INT +2 per 10 Character Levels', 'LUK +2 per 10 Character Levels', 'ATT +1 per 10 Character Levels', 'MATT +1 per 10 Character Levels', 'Status Resistance +5', 'Ignore Enemy Defense +5%', 'Damage when attacking boss monsters +18%']
Unique
['Max HP +7%', 'Max MP +7%', 'ATT +9%', 'MATT +9%', 'Critical Chance +9%', 'STR +9%', 'DEX +9%', 'INT +9%', 'LUK +9%', 'Damage +9%', 'All Stats +6%', 'STR +1 per 10 Character Levels', 'DEX +1 per 10 Character Levels', 'INT +1 per 10 Character Levels', 'LUK +1 per 10 Character Levels', '15% chance to recover 95 HP when attacking', '15% chance to recover 95 MP when attacking', 'Ignore Enemy Defense +4%', 'Damage when attacking boss monsters +12%']


## Picking a line from Legend or unique

In [6]:
import random

In [7]:
def cube(prime_rate):
    equip_line=[]
    for i in prime_rate:
        if random.randint(1, 1000000)<=int(i*10000):
            equip_line+=random.choices(list(Lines[main_line].Line), weights=tuple(Lines[main_line].Weight), k=1)# generate main line
        else:
            equip_line+=random.choices(list(Lines[sub_line].Line), weights=tuple(Lines[sub_line].Weight), k=1)# generate secondary line
    return equip_line

In [8]:
"""Test cubes"""
df=[] #df represent the full data
for x in range(100000): 
    result=cube(prime_rates) #result is a list
    i=0
    for y in result: #y is a string
            if y.startswith(desired_lines):
                i+=1
    result.append(i)
    df.append(result)
del x,y
df=pd.DataFrame(df,columns=column_header) 
# df1=df[df.line1.str.startswith("C")]
df1=df[df.desired>=desired_number]
df.to_csv(r'all lines.csv')
df1.to_csv(r"desired.csv")


In [9]:
print(df.shape)
print(df1.shape)
print(df1.shape[0]/df.shape[0]*100,"%")

(100000, 4)
(869, 4)
0.869 %
